<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=d19ac3aef8f375c038cfad44573303c5d07a0e04c35efe9cfed15fd239a5bc73
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1
    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-21 09:09:08
-------------------
qualified stocks: 87
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  400.00
Current:  1.45 C
-------------------
Today PnL: 0.00 (0.0%)
Current PnL: -21.20 L (-13.89%)
CY Booked + Current PnL: -7.31 L (-4.79%)
-------------------
Total profit:  1.78 L
Total loss:  -22.98 L
-------------------
Total Booked + Current PnL: 19.67 L (15.66%)
Total Booked PnL: 40.87 L (32.53%)
Curr Year Booked PnL: 13.89 L (9.61%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.31 C
Est FTT PnL: 86.61 L (59.9%)
Deployed:  1.26 C
Current:  1.45 C
CAGR/XIRR %: 8.07%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,0.0,-7.16,7.74,0.02,12543.0,-12503.0,162056.0,147.21,77.0,M-SC,4.32,234.0,-1.00,1.12,40.96,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,-0.0,-14.23,16.69,0.09,14427.0,-14337.0,86440.0,100.64,49.0,M-SC,4.15,253.0,-0.99,0.60,13.56,OX40N,NTT,DURABLES
83,VOLTAS,1530.00,0.0,10.45,8.37,19.69,17725.0,20028.0,211770.0,-0.39,58.0,X-MC,3.60,78.0,1.13,1.46,17.95,XY25,NTT,AC
35,ICICIGI,2252.93,0.0,8.71,10.57,20.19,20030.0,15177.0,189497.0,-14.84,57.0,X-MC,7.75,68.0,0.76,1.31,25.14,X40,ATH,INSURANCE
43,ITC,452.00,-0.0,-0.94,11.48,10.44,22761.0,-1873.0,198265.0,-40.74,45.0,X-LC,0.97,5.0,-0.08,1.37,4.58,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
0,5PAISA,593.00,0.0,-32.11,66.29,12.90,98339.0,-70150.0,148346.0,118.75,68.0,H-SC,14.93,161.0,-0.71,1.03,23.35,OX40N,NTT,FINANCE
1,ABB,7934.00,0.0,-5.57,54.15,45.57,133781.0,-14563.0,247056.0,-39.35,53.0,H-MC,5.45,120.0,-0.11,1.71,5.12,AR,NTT,ELECTRICAL
2,ABBOTINDIA,35195.00,0.0,-1.28,18.96,17.44,28047.0,-1922.0,147925.0,-14.93,48.0,X-MC,4.12,63.0,-0.07,1.02,17.63,X40,ATH,PHARMA
3,ACC,3906.00,0.0,-22.19,110.98,64.15,205468.0,-52811.0,185140.0,-54.57,51.0,X-SC,1.79,82.0,-0.26,1.28,3.85,XY24,BTT,CEMENT
4,ALKYLAMINE,4546.37,-0.0,-24.38,156.06,93.63,119146.0,-24617.0,76346.0,-19.07,37.0,H-SC,4.98,145.0,-0.21,0.53,13.72,SR,ATH,CHEMICALS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
0,5PAISA,593.00,0.0,-32.11,66.29,12.90,98339.0,-70150.0,148346.0,118.75,68.0,H-SC,14.93,161.0,-0.71,1.03,23.35,OX40N,NTT,FINANCE
1,ABB,7934.00,0.0,-5.57,54.15,45.57,133781.0,-14563.0,247056.0,-39.35,53.0,H-MC,5.45,120.0,-0.11,1.71,5.12,AR,NTT,ELECTRICAL
2,ABBOTINDIA,35195.00,0.0,-1.28,18.96,17.44,28047.0,-1922.0,147925.0,-14.93,48.0,X-MC,4.12,63.0,-0.07,1.02,17.63,X40,ATH,PHARMA
3,ACC,3906.00,0.0,-22.19,110.98,64.15,205468.0,-52811.0,185140.0,-54.57,51.0,X-SC,1.79,82.0,-0.26,1.28,3.85,XY24,BTT,CEMENT
4,ALKYLAMINE,4546.37,-0.0,-24.38,156.06,93.63,119146.0,-24617.0,76346.0,-19.07,37.0,H-SC,4.98,145.0,-0.21,0.53,13.72,SR,ATH,CHEMICALS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.0,-2.80,113.90,107.91,163461.0,-4131.0,143513.0,-20.96,39.0,M-SC,10.15,216.0,-0.03,0.99,0.12,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,0.0,-7.16,7.74,0.02,12543.0,-12503.0,162056.0,147.21,77.0,M-SC,4.32,234.0,-1.00,1.12,40.96,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,-0.0,-14.23,16.69,0.09,14427.0,-14337.0,86440.0,100.64,49.0,M-SC,4.15,253.0,-0.99,0.60,13.56,OX40N,NTT,DURABLES
47,KANSAINER,340.00,0.0,-19.97,41.80,13.47,90205.0,-53865.0,215802.0,-67.15,35.0,H-SC,1.70,158.0,-0.60,1.49,9.73,XY24,NTT,PAINTS
73,TATAELXSI,9161.00,0.0,-22.66,70.71,32.03,72099.0,-29871.0,101964.0,-16.37,50.0,H-SC,6.47,148.0,-0.41,0.71,14.21,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,0.0,1.01,70.55,72.27,120747.0,1703.0,171151.0,-13.34,56.0,M-LC,5.02,99.0,0.01,1.18,7.24,XR,NTT,IT
38,INDIAMART,4810.62,-0.0,2.78,101.13,106.72,128197.0,3429.0,126765.0,-49.10,42.0,H-SC,4.52,139.0,0.03,0.88,26.81,AR,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,-0.0,2.78,101.13,106.72,128197.0,3429.0,126765.0,-49.10,42.0,H-SC,4.52,139.0,0.03,0.88,26.81,AR,ATH,MISC
85,WIPRO,420.00,0.0,1.01,70.55,72.27,120747.0,1703.0,171151.0,-13.34,56.0,M-LC,5.02,99.0,0.01,1.18,7.24,XR,NTT,IT
25,FINCABLES,1641.55,-0.0,-2.80,113.90,107.91,163461.0,-4131.0,143513.0,-20.96,39.0,M-SC,10.15,216.0,-0.03,0.99,0.12,OX40N,ATH,CABLES
84,WHIRLPOOL,2270.00,0.0,-4.80,91.09,81.92,99550.0,-5511.0,109287.0,-44.29,32.0,M-SC,6.21,236.0,-0.06,0.76,29.38,XR,NTT,DURABLES
86,ZYDUSLIFE,1286.17,0.0,-2.14,38.60,35.63,79164.0,-4486.0,205088.0,-15.85,33.0,H-MC,4.77,119.0,-0.06,1.42,14.17,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TMPV,600.00,0.0,-14.74,66.76,42.18,157333.0,-40741.0,235669.0,-23.08,20.0,X-LC,0.13,3.0,-0.26,1.63,0.00,XY24,NTT,AUTO
11,BATAINDIA,2096.00,-0.0,-34.89,107.65,35.20,90189.0,-44890.0,83780.0,9.89,23.0,X-SC,12.05,92.0,-0.50,0.58,0.00,X40,NTT,FOOTWEAR
34,HONAUT,58357.33,0.0,-14.82,68.25,43.32,94690.0,-24132.0,138740.0,-26.83,27.0,X-SC,1.66,90.0,-0.25,0.96,6.53,X40N,ATH,ELECTRICAL
46,JSWINFRA,342.00,0.0,-6.93,23.80,15.23,44380.0,-13878.0,186469.0,-20.37,29.0,X-MC,6.32,66.0,-0.31,1.29,23.09,X40N,NTT,REALTY
54,PAGEIND,51605.08,0.0,-5.79,33.81,26.07,52155.0,-9480.0,154260.0,-30.68,31.0,X-MC,8.72,55.0,-0.18,1.07,0.00,X40,ATH,APPARELS


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TMPV,600.00,0.0,-14.74,66.76,42.18,157333.0,-40741.0,235669.0,-23.08,20.0,X-LC,0.13,3.0,-0.26,1.63,0.00,XY24,NTT,AUTO
43,ITC,452.00,-0.0,-0.94,11.48,10.44,22761.0,-1873.0,198265.0,-40.74,45.0,X-LC,0.97,5.0,-0.08,1.37,4.58,X40,NTT,FMCG
36,ICICIPRULI,790.00,-0.0,2.52,27.75,30.97,50281.0,4453.0,181191.0,-21.60,55.0,X-MC,1.53,75.0,0.09,1.25,15.40,X40,ATH,INSURANCE
34,HONAUT,58357.33,0.0,-14.82,68.25,43.32,94690.0,-24132.0,138740.0,-26.83,27.0,X-SC,1.66,90.0,-0.25,0.96,6.53,X40N,ATH,ELECTRICAL
3,ACC,3906.00,0.0,-22.19,110.98,64.15,205468.0,-52811.0,185140.0,-54.57,51.0,X-SC,1.79,82.0,-0.26,1.28,3.85,XY24,BTT,CEMENT


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,-0.0,-34.89,107.65,35.20,90189.0,-44890.0,83780.0,9.89,23.0,X-SC,12.05,92.0,-0.50,0.58,0.00,X40,NTT,FOOTWEAR
54,PAGEIND,51605.08,0.0,-5.79,33.81,26.07,52155.0,-9480.0,154260.0,-30.68,31.0,X-MC,8.72,55.0,-0.18,1.07,0.00,X40,ATH,APPARELS
76,TMPV,600.00,0.0,-14.74,66.76,42.18,157333.0,-40741.0,235669.0,-23.08,20.0,X-LC,0.13,3.0,-0.26,1.63,0.00,XY24,NTT,AUTO
9,BAJAJHFL,181.50,-0.0,-12.96,70.57,48.45,123153.0,-25994.0,174512.0,-19.23,33.0,X-MC,7.25,64.0,-0.21,1.21,0.63,X40N,ATH,FINANCE
19,COLPAL,3726.84,0.0,-17.24,70.98,41.51,154715.0,-45395.0,217970.0,-1.46,44.0,X-MC,7.19,61.0,-0.29,1.51,2.29,XY25,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.00,-0.0,-27.30,96.49,42.84,45599.0,-17748.0,47258.0,-52.62,39.0,X-SC,37.08,83.0,-0.39,0.33,2.40,XY24,NTT,MISC
60,RELAXO,1176.00,0.0,-46.35,183.89,52.31,143212.0,-67281.0,77879.0,-42.89,33.0,X-SC,3.38,91.0,-0.47,0.54,4.25,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.0,-34.89,107.65,35.20,90189.0,-44890.0,83780.0,9.89,23.0,X-SC,12.05,92.0,-0.50,0.58,0.00,X40,NTT,FOOTWEAR
52,MEDANTA,1486.00,0.0,1.29,22.50,24.08,28933.0,1639.0,128589.0,-6.93,41.0,X-SC,4.54,89.0,0.06,0.89,21.16,XY24,NTT,HEALTHCARE
22,DIXON,18931.72,0.0,-1.66,23.63,21.58,32566.0,-2322.0,137817.0,-50.44,41.0,X-MC,7.15,56.0,-0.07,0.95,22.01,X40N,ATH,IT


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4389.97,-0.0,-14.55,39.59,19.28,117032.0,-50345.0,295611.0,-25.57,63.0,X-LC,7.78,1.0,-0.43,2.04,9.28,X40,ATH,IT
41,INFY,2275.00,-0.0,4.60,48.06,54.87,157288.0,14386.0,327274.0,-18.52,58.0,X-LC,3.33,2.0,0.09,2.26,13.26,X40,BTT,IT
76,TMPV,600.00,0.0,-14.74,66.76,42.18,157333.0,-40741.0,235669.0,-23.08,20.0,X-LC,0.13,3.0,-0.26,1.63,0.00,XY24,NTT,AUTO
82,VBL,671.64,0.0,-8.80,48.76,35.67,140457.0,-27785.0,288057.0,-19.67,43.0,X-LC,6.11,4.0,-0.20,1.99,3.84,X40N,ATH,FMCG
43,ITC,452.00,-0.0,-0.94,11.48,10.44,22761.0,-1873.0,198265.0,-40.74,45.0,X-LC,0.97,5.0,-0.08,1.37,4.58,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COFFEEDAY,80.00,0.0,-37.61,116.80,35.27,82750.0,-42701.0,70848.0,-51.71,34.0,L-SC,18.89,270.0,-0.52,0.49,73.40,XR,NTT,HOTELS
6,ASIANTILES,137.00,0.0,-11.28,104.78,81.67,87202.0,-10586.0,83224.0,7483.33,57.0,L-SC,18.49,271.0,-0.12,0.58,61.17,XR,NTT,CERAMICS
68,SONACOMS,806.63,0.0,-12.18,58.89,39.54,52317.0,-12320.0,88839.0,-31.30,70.0,M-SC,6.90,220.0,-0.24,0.61,25.27,AR,ATH,AUTO
51,MASFIN,398.61,0.0,-2.53,25.21,22.05,24077.0,-2475.0,95505.0,-15.99,59.0,H-SC,7.08,164.0,-0.10,0.66,38.82,XR,ATH,FINANCE
65,SHALBY,327.00,-0.0,-14.99,47.15,25.10,72610.0,-27145.0,153998.0,1032.38,36.0,M-SC,16.69,250.0,-0.37,1.07,23.41,XY24,NTT,HEALTHCARE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,0.0,-7.16,7.74,0.02,12543.0,-12503.0,162056.0,147.21,77.0,M-SC,4.32,234.0,-1.00,1.12,40.96,OX40N,NTT,PAINTS
13,BSOFT,831.70,0.0,-29.51,113.91,50.79,112935.0,-41509.0,99144.0,-9.55,59.0,H-SC,7.62,151.0,-0.37,0.69,15.44,XR,ATH,IT
0,5PAISA,593.00,0.0,-32.11,66.29,12.90,98339.0,-70150.0,148346.0,118.75,68.0,H-SC,14.93,161.0,-0.71,1.03,23.35,OX40N,NTT,FINANCE
5,ASIANPAINT,3460.25,-0.0,-0.04,21.00,20.95,52849.0,-106.0,251662.0,-1.05,75.0,X-LC,12.08,31.0,-0.00,1.74,35.58,X40,ATH,PAINTS
68,SONACOMS,806.63,0.0,-12.18,58.89,39.54,52317.0,-12320.0,88839.0,-31.30,70.0,M-SC,6.90,220.0,-0.24,0.61,25.27,AR,ATH,AUTO


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.73
1,25,44.52
2,50,75.34


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.81
MC    29.23
LC    24.95
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.98
X40      21.01
X40N     11.86
XR        9.62
XY25      9.06
AR        9.03
OX40N     7.74
X200      1.88
SR        1.00
MH        0.81
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.18
X-MC    22.83
X-LC    19.81
M-SC    12.06
X-SC     8.15
H-MC     4.70
H-LC     2.99
L-SC     1.42
M-MC     1.37
M-LC     1.18
L-LC     0.97
L-MC     0.33
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.71,-4.12,37.79
IT,13.79,-13.39,71.92
FINANCE,9.48,-13.17,60.99
MISC,7.46,-20.22,69.47
PAINTS,6.00,-6.31,22.11
ELECTRICAL,5.88,-10.32,50.73
INSURANCE,4.46,2.09,32.41
PHARMA,3.83,-0.90,33.19
AUTO,3.32,-19.68,71.00


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2992090.0,21
XR,1272366.0,13
AR,1266341.0,10
X40,998228.0,14
X40N,732305.0,9
OX40N,673881.0,10
XY25,330150.0,6
SR,275559.0,2
MH,67093.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3484177.0,25
M-SC,1316566.0,15
X-MC,1179449.0,16
X-LC,896380.0,11
X-SC,741508.0,8
H-MC,399102.0,3
L-SC,256567.0,3
H-LC,125613.0,2
M-LC,120747.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1193785.0      6
           AR         858964.0      5
           XR         778853.0      7
M-SC       XY24       770052.0      6
X-MC       X40        430602.0      7
X-LC       X40        393355.0      5
X-MC       XY24       337115.0      3
H-SC       OX40N      309923.0      4
X-SC       X40N       287237.0      3
           XY24       280000.0      3
M-SC       OX40N      277343.0      5
H-SC       SR         275559.0      2
X-MC       X40N       239292.0      4
X-LC       XY24       224981.0      2
H-MC       AR         212945.0      2
X-LC       X40N       205776.0      2
H-MC       XY24       186157.0      1
X-SC       X40        174271.0      2
X-MC       XY25       172440.0      2
L-SC       XR         169952.0      2
M-SC       XR         147492.0      2
           AR         121679.0      2
M-LC       XR         120747.0      1
L-SC       OX40N       86615.0      1
H-LC       AR          72753.0      1
X-LC       XY25        72268.0      2
H-SC       MH          67093.0      1
L-MC       XR          55322.0      1
H-LC       X200        52860.0      1
M-MC       XY25        46532.0      1
L-LC       XY25        38910.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 64.0 seconds
